In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import torch
import numpy as np
import time
from torchvision import datasets, transforms
from scipy.io import savemat, loadmat
from torch.utils.data import DataLoader, Subset, TensorDataset
from aeon.datasets import load_classification
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
from tqdm import tqdm


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset_name = 'WalkingSittingStanding'

In [3]:
X_train, y_train, metadata = load_classification(dataset_name, return_metadata=True, split='train')
X_test, y_test = load_classification(dataset_name, split='test')
if X_train.shape[0] < 200:
    if X_test.shape[0] >= 200:
        train_size = int((X_train.shape[0] + X_test.shape[0]) * 1/4)
        x, y = load_classification(dataset_name)
        X_train, y_train = x[:train_size, :], y[:train_size]
        X_test, y_test = x[train_size:, :], y[train_size:]

In [4]:
input_channels = 1
if X_train.ndim == 3:
    input_channels = X_train.shape[1]
seq_length = X_train.shape[-1]
if y_train.dtype == object or isinstance(y_train[0], str):
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_test = le.transform(y_test)

In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1))
X_test_scaled = scaler.transform(X_test.reshape(X_test.shape[0], -1))


X_min = X_train_scaled.min(axis=0)
X_max = X_train_scaled.max(axis=0)

denom = (X_max - X_min)
denom[denom == 0] = 1   # avoid division by zero

X_train_norm = (X_train_scaled - X_min) / denom
X_test_norm  = (X_test_scaled  - X_min) / denom

# Optional: clip to [0,1] just in case
X_train_norm = np.clip(X_train_norm, 0, 1)
X_test_norm  = np.clip(X_test_norm, 0, 1)
X_train_tensor = torch.tensor(X_train_norm, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test_norm, dtype=torch.float32).to(device)

In [6]:
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

num_classes = len(np.unique(y_train))

In [7]:
# 1. Generate bipolar lookup table
def lookup_generate(dim: int, datatype: str, n_keys: int, device: torch.device):
    if datatype != 'bipolar':
        raise ValueError("Only 'bipolar' supported")
    tbl = torch.randint(0, 2, (n_keys, dim), device=device, dtype=torch.int8)
    return tbl * 2 - 1  # map {0,1} → {-1,+1}

# 2. Encode a batch of inputs into hypervectors
@torch.no_grad()
def encode_batch(X: torch.Tensor, position_table: torch.Tensor, grayscale_table: torch.Tensor):
    n_keys = grayscale_table.shape[0]
    X_normalized = ((X + 3) / 6) * (n_keys - 1)  # map [-3,3] → [0, n_keys-1]
    X_indices = X_normalized.round().clamp(0, n_keys - 1).long()  # indices in [0, n_keys-1]
    gray = grayscale_table[X_indices]            # (N, D_in, dim)
    pos  = position_table.unsqueeze(0)           # (1, D_in, dim)
    hv   = (pos * gray).sum(dim=1)               # (N, dim)
    return hv

# 3. Train associative memory by summing all encodings per class
def train_am(X_train, Y_train, position_table, grayscale_table, dim: int):
    H_train = encode_batch(X_train, position_table, grayscale_table).float()  # (N, dim)
    C = int(Y_train.max().item()) + 1
    am = torch.zeros((C, dim), device=X_train.device, dtype=torch.float32)
    am = am.index_add(0, Y_train, H_train)
    return am

# 4. Single-image prediction (returns class and query HV)
@torch.no_grad()
def predict_(am, img, position_table, grayscale_table):
    qhv = encode_batch(img.unsqueeze(0), position_table, grayscale_table).squeeze(0).float()
    sims = F.cosine_similarity(qhv.unsqueeze(0), am, dim=1)  # (C,)
    pred = int(sims.argmax().item())
    return pred, qhv

def predict(am, img, position_table, grayscale_table):
    pred, _ = predict_(am, img, position_table, grayscale_table)
    return pred

# 5. Test on full set
@torch.no_grad()
def test(am, X_test, Y_test, position_table, grayscale_table):
    H_test = encode_batch(X_test, position_table, grayscale_table).float()  # (N_test, dim)
    h_norm = H_test.norm(dim=1, keepdim=True)                              # (N,1)
    a_norm = am.norm(dim=1, keepdim=True).t()                              # (1,C)
    sims   = (H_test @ am.t()) / (h_norm * a_norm)                         # (N,C)
    preds  = sims.argmax(dim=1)                                            # (N,)
    acc    = (preds == Y_test).float().mean().item()
    print(f"Testing accuracy: {acc:.4f}")
    return acc

# 6. Load a saved model (AM + tables)
def loadmodel(fpath: str, device: torch.device = None):
    with open(fpath, 'rb') as f:
        am_np, pos_np, gray_np = pickle.load(f)
    am   = torch.from_numpy(am_np)
    pos  = torch.from_numpy(pos_np)
    gray = torch.from_numpy(gray_np)
    if device is not None:
        am, pos, gray = am.to(device), pos.to(device), gray.to(device)
    return am, pos, gray

# 7. Quantize the AM to a lower bit-width
def quantize(am: torch.Tensor, before_bw: int, after_bw: int) -> torch.Tensor:
    if before_bw <= after_bw:
        return am.clone()
    shift = before_bw - after_bw
    return torch.round(am.float() / (2 ** shift)).to(am.dtype)

# 8. Batched AM training
@torch.no_grad()
def train_am_batched(
    X_train: torch.Tensor,
    Y_train: torch.Tensor,
    position_table: torch.Tensor,
    grayscale_table: torch.Tensor,
    dim: int,
    batch_size: int = 128,
    device: torch.device = None
) -> torch.Tensor:
    N = X_train.shape[0]
    C = int(Y_train.max().item()) + 1
    am = torch.zeros(C, dim, device=device, dtype=torch.float32)
    for i in (range(0, N, batch_size)):
        xb = X_train[i : i + batch_size]
        yb = torch.as_tensor(Y_train[i : i + batch_size], device=device)  # <== ✅ fixed
        hb = encode_batch(xb, position_table, grayscale_table).float()
        am = am.index_add(0, yb, hb)
    return am

# 9. Test on a split (non-batched)
@torch.no_grad()
def test_split(am, X_split, Y_split, position_table, grayscale_table):
    Hs   = encode_batch(X_split, position_table, grayscale_table).float()  # (M, dim)
    sims = F.cosine_similarity(Hs.unsqueeze(1), am.unsqueeze(0), dim=2)   # (M, C)
    preds = sims.argmax(dim=1)                                            # (M,)
    return (preds == Y_split).float().mean().item()

@torch.no_grad()
def flip_rows_(tensor: torch.Tensor, perc: float) -> torch.Tensor:
    """
    In-place sign flip of `perc` fraction of elements in *every* row.

    tensor : (B, D)  – any real dtype (+1/-1 HVs work fine)
    perc   : 0‒1     – fraction of dimensions to flip per row

    Returns the same tensor object (for chaining).
    """
    if not 0.0 <= perc <= 1.0:
        raise ValueError("perc must be in [0,1]")

    # trivial cases
    if perc == 0.0 or tensor.numel() == 0:
        return tensor

    B, D = tensor.shape
    k = int(round(D * perc))            # exact #dims to flip / row
    if k == 0:
        return tensor

    device = tensor.device
    # --- vectorised selection of k unique indices per row -------------
    # 1) random scores per entry
    rnd = torch.rand(B, D, device=device)
    # 2) take indices of the largest k scores along each row
    _, idx = rnd.topk(k, dim=1, largest=True, sorted=False)  # (B,k)
    # 3) convert (row, col) pairs → flat indices
    base = torch.arange(B, device=device).unsqueeze(1) * D   # (B,1)
    flat_idx = (idx + base).reshape(-1)                      # (B*k,)
    # 4) flip in-place
    tensor.view(-1)[flat_idx] *= -1
    return tensor
    
# 11. Test on a split (batched)
@torch.no_grad()
def test_split_batched(
    am: torch.Tensor,
    X: torch.LongTensor,
    Y: torch.LongTensor,
    position_table: torch.Tensor,
    grayscale_table: torch.Tensor,
    encode_fn,
    flip_perc=0.0,
    batch_size: int = 128,
    device: torch.device = None
) -> float:
    correct, total = 0, 0
    for i in range(0, X.size(0), batch_size):
        xb = X[i : i + batch_size].to(device)
        yb = Y[i : i + batch_size].to(device)
        hb = encode_fn(xb, position_table, grayscale_table).float()
        flip_rows_(hb, perc=flip_perc)
        sims  = F.cosine_similarity(hb.unsqueeze(1), am.unsqueeze(0), dim=2)
        preds = sims.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total   += yb.size(0)
    return correct / total

In [8]:
D = 15000

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# n_rounds   = 20
n_splits = 20
split_size = len(test_data) // n_splits
flip_percs = np.arange(0.0, 0.51, 0.05)
accuracies = np.zeros((len(flip_percs), n_splits))
n_class    = num_classes
q_bit      = 16
print(split_size)

147


In [10]:
X_train.shape, X_test.shape

((7352, 3, 206), (2947, 3, 206))

In [11]:
for i, perc in enumerate(flip_percs):
    print(f"\n==> Flip Percentage: {perc}")
    for split_idx in range(n_splits):
        # Dynamically get input dimension
        input_dim = X_train_tensor.shape[1] 
        n_keys = 64                   
        position_table  = lookup_generate(D, 'bipolar', input_dim, device=device)
        grayscale_table = lookup_generate(D, 'bipolar', n_keys, device=device)
        am = train_am_batched(
            X_train_tensor, y_train_tensor,
            position_table, grayscale_table,
            dim=D,
            batch_size=1,
            device=device
        )
        indices = list(range(len(X_test_tensor)))
        np.random.shuffle(indices)  # or random.shuffle(indices)
        start = split_idx * split_size
        end   = start + split_size
        split_indices = indices[start:end]
        am_q = quantize(am, before_bw=16, after_bw=q_bit)
        acc = test_split_batched(
            am_q,
            X_test_tensor[split_indices],
            y_test_tensor[split_indices],
            # X_test_tensor,
            # y_test_tensor,
            position_table,
            grayscale_table,
            encode_batch,  
            flip_perc=perc,
            batch_size=10,
            device=device
        )
        accuracies[i, split_idx] = acc*100

    print("Accuracy average for 20 rounds:", accuracies[i].mean())

    # Free GPU memory
    del position_table, grayscale_table, am, am_q
    torch.cuda.empty_cache()



==> Flip Percentage: 0.0
Accuracy average for 20 rounds: 69.93197278911565

==> Flip Percentage: 0.05
Accuracy average for 20 rounds: 69.8639455782313

==> Flip Percentage: 0.1
Accuracy average for 20 rounds: 68.74149659863946

==> Flip Percentage: 0.15000000000000002
Accuracy average for 20 rounds: 69.69387755102039

==> Flip Percentage: 0.2
Accuracy average for 20 rounds: 71.05442176870747

==> Flip Percentage: 0.25
Accuracy average for 20 rounds: 70.54421768707483

==> Flip Percentage: 0.30000000000000004
Accuracy average for 20 rounds: 67.51700680272108

==> Flip Percentage: 0.35000000000000003
Accuracy average for 20 rounds: 68.02721088435374

==> Flip Percentage: 0.4
Accuracy average for 20 rounds: 67.44897959183672

==> Flip Percentage: 0.45
Accuracy average for 20 rounds: 63.46938775510205

==> Flip Percentage: 0.5
Accuracy average for 20 rounds: 19.14965986394558


In [12]:
np.mean(accuracies, axis=1)

array([69.93197279, 69.86394558, 68.7414966 , 69.69387755, 71.05442177,
       70.54421769, 67.5170068 , 68.02721088, 67.44897959, 63.46938776,
       19.14965986])

In [13]:
savemat(f'{dataset_name}_VanillaHDC.mat', {f'{dataset_name}_VanillaHDC': accuracies})

In [14]:
np.std(accuracies, axis=1)

array([2.67477595, 3.80648567, 3.69153407, 3.23737344, 4.20160969,
       4.02511896, 3.29686422, 4.7277688 , 2.36901771, 4.39868385,
       2.95174607])